In [57]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from Parser import parse_frames
from scipy.signal import medfilt
from Functions import analyse_each_rep, find_extremas
from JointAngles import JointAngles
import numpy as np
import glob
import os
import pandas as pd
from functools import reduce

In [114]:
input_folder = "C:\\Users\\altaok\\Documents\\GitHub\\IndividualProject\\keypoints_for_all"
folder_paths = glob.glob(os.path.join(input_folder, 'output_points_bicep*'))


def fill_dataframe(keypoints_paths):
    angle_label_dict = {}
    data = {}
    labels = []
    df_list = []
    for folder in folder_paths:
        video_name = os.path.basename(folder).replace('output_points_', '')
        label = 0 if 'correct' in folder else 1
        frame_poses = parse_frames(folder)
        joint_angles = JointAngles('bicep curl', frame_poses)

        upArm_trunk_angles = np.array(joint_angles.upArm_trunk_angles, dtype=np.float)
        upArm_trunk_angles_filtered = medfilt(medfilt(upArm_trunk_angles, 5), 5)

        upArm_forearm_angles = np.array(joint_angles.upArm_forearm_angles, dtype=np.float)
        upArm_forearm_angles_filtered = medfilt(medfilt(upArm_forearm_angles, 5), 5)

        trunk_knee_angles = np.array(joint_angles.trunk_knee_angles, dtype=np.float)
        trunk_knee_angles_filtered = medfilt(medfilt(trunk_knee_angles, 5), 5)
        
        #number of reps 
        upArm_forearm_maximas, indexesmax3 = find_extremas(upArm_forearm_angles_filtered) 
       
        if upArm_forearm_maximas.size == 0:
            s1 = pd.Series([upArm_forearm_angles_filtered, upArm_trunk_angles_filtered,  trunk_knee_angles_filtered], name='Angle_array')
            s2 = pd.Series([label, label, label], name='Label')
            df = pd.concat([s1,s2], axis=1)           
      
            df_list.append(df)
           
            
        else:         
            # Extract rep angles
            each_rep_angles = analyse_each_rep(trunk_knee_angles_filtered, upArm_forearm_angles_filtered, upArm_trunk_angles_filtered, upArm_forearm_maximas)            
            for n in each_rep_angles:
                labels.append(label)    
                        
            s1 = pd.Series(each_rep_angles, name='Angle_array')
            s2 = pd.Series(labels, name='Label')
            df = pd.concat([s1,s2], axis=1)
        
            df_list.append(df)
            # Erase labels
            labels = []
            
    
    if len(df_list) > 1:
        df_final = pd.concat(df_list, axis=0).reset_index(drop=True)
       
    return df_final


        

In [115]:
df = fill_dataframe(folder_paths)
print(df.info())
print(df.head())

    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
Angle_array    33 non-null object
Label          33 non-null int64
dtypes: int64(1), object(1)
memory usage: 656.0+ bytes
None
                                         Angle_array  Label
0  [170.7536123592761, 170.7536123592761, 170.753...      0
1  [0.10820696782771888, 0.11090423210451548, 0.2...      0
2  [176.66379719540063, 177.34931455996818, 177.6...      0
3  [132.88076293755415, 132.88076293755415, 132.8...      0
4  [5.534939872641024, 5.534939872641024, 5.53493...      0


#   Dataset Cleaning

In [116]:
df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
Angle_array    33 non-null object
Label          33 non-null int64
dtypes: int64(1), object(1)
memory usage: 656.0+ bytes


In [117]:
t = df['Angle_array']

for n in t:
    print(n.size)

99
99
99
66
66
66
69
69
69
74
74
74
66
66
66
68
68
68
70
70
70
70
70
70
65
65
65
112
112
112
88
88
88


In [129]:
from sklearn.model_selection import train_test_split
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from sklearn import metrics

y = df.Label
x = df.drop('Label', axis=1)


In [119]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
X_train.head()

,Angle_array
0,"[170.7536123592761, 170.7536123592761, 170.753..."
19,"[7.00293838955028, 7.706088080838667, 7.706088..."
13,"[5.496729160731521, 5.496729160731521, 5.49672..."
22,"[3.14273656839872, 3.14273656839872, 2.5123444..."
11,"[176.52866111078774, 176.4652238755618, 175.93..."


In [127]:
X_train.shape

(26, 1)

In [123]:
y_train.shape

(26,)

In [121]:
X_test.shape

(7, 1)

In [125]:
k_range = range(1, 26)
scores = {}
scores_list = []
for k in k_range:
    knn_clf = KNeighborsTimeSeriesClassifier(n_neighbors=k, metric="dtw")
    knn_clf.fit(X_train, y_train)
    predicted_labels = knn_clf.predict(X_test)
    acc = metrics.accuracy_score(y_test, predicted_labels)
    scores[k] = acc
    print("Correct classification rate:", acc)


ValueError: setting an array element with a sequence.